# Project Proposal (Matt)

ADD TEXT HERE

# Project Code

## Data Extraction

### Flight Data (Myles & Sedra)

MK NOTES
1. Flight Data - Columns to Drop: 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'
2. Flight Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
3. Flight Data - Add Column for Total Delay (Add Together Departure & Arrival Delay Values)
4. Flight Data - Combine Date Value Columns into Single Date Column
5. Airline Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
6. Airport Data - Columns to Drop: 'LATITUDE', 'LONGITUDE'

In [ ]:
#Import Modules
import os
import pandas as pd
import numpy as np
import datetime

In [ ]:
flight_path = "Flight data/flights.csv"
airline_path = "Flight data/airlines.csv"
airport_path = "Flight data/airports.csv"

In [ ]:
flights_pd = pd.read_csv(flight_path)
flights_pd.head()

In [ ]:
airport_pd = pd.read_csv(airport_path)
airport_pd.head()

In [ ]:
airline_pd = pd.read_csv(airline_path)
airline_pd.head()

In [ ]:
#Flight Data - Drop: 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'
update1_flights = flights_pd.drop(columns=['CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'])
update1_flights.head()

In [ ]:
#Flight Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
array = ['UA', 'AA', 'US', 'WN', 'DL', 'VX']
update2_flights = update1_flights.loc[update1_flights['AIRLINE'].isin(array)]
update2_flights

In [ ]:
update2_flights['Total Delay'] = update1_flights['SCHEDULED_DEPARTURE'] + update1_flights['ARRIVAL_DELAY']
update2_flights.head()

In [ ]:
#Flight Data - Combine Date Value Columns into Single Date 


In [ ]:
############----------------------------------------------------------------------------------------------

In [ ]:
#Airline Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
array = ['UA', 'AA', 'US', 'WN', 'DL', 'VX']
update1_airline = airline_pd.loc[airline_pd['IATA_CODE'].isin(array)]
update1_airline

In [ ]:
#########-----Airport Extraction

In [ ]:
# Airport Data - Columns to Drop: 'LATITUDE', 'LONGITUDE'
update1_airport = airport_pd.drop(columns=['LATITUDE', 'LONGITUDE'])
update1_airport.head()

In [ ]:
#FINAL DF
update3_flights = update2_flights[(update2_flights['DAY'] >= 16) & (update2_flights['DAY'] <= 24)]
update3_flights.head()

In [ ]:
###-----------------------------------------------------------------------------------

In [ ]:
latedf = update3_flights[update3_flights['Total Delay']>0]

scheduled_arrival = []
for item in latedf['SCHEDULED_ARRIVAL'].astype(int):
    if item == 2400:
        scheduled_arrival.append(datetime.datetime.strptime('0000', '%H%M').strftime('%H:%M'))
    elif item >= 1000:
        scheduled_arrival.append(datetime.datetime.strptime(str(item), '%H%M').strftime('%H:%M'))
    elif item >= 100 and item <= 999:
        scheduled_arrival.append(datetime.datetime.strptime('0' + str(item), '%H%M').strftime('%H:%M'))
    elif item >= 10 and item <= 99:
        scheduled_arrival.append(datetime.datetime.strptime('00'+ str(item), '%H%M').strftime('%H%M'))
    elif item <=9 :
        scheduled_arrival.append(datetime.datetime.strptime('000'+str(item), '%H%M').strftime('%H%M'))

scheduled_departure = []
for item in latedf['SCHEDULED_DEPARTURE'].astype(int):
    if item == 2400:
        scheduled_departure.append(datetime.datetime.strptime('0000', '%H%M').strftime('%H:%M'))
    elif item >= 1000:
        scheduled_departure.append(datetime.datetime.strptime(str(item), '%H%M').strftime('%H:%M'))
    elif item >= 100 and item <= 999:
        scheduled_departure.append(datetime.datetime.strptime('0' + str(item), '%H%M').strftime('%H:%M'))
    elif item >= 10 and item <= 99:
        scheduled_departure.append(datetime.datetime.strptime('00'+ str(item), '%H%M').strftime('%H%M'))
    elif item <=9 :
        scheduled_departure.append(datetime.datetime.strptime('000'+str(item), '%H%M').strftime('%H%M'))
        
departure_time = []
for item in latedf['DEPARTURE_TIME'].astype(int):
    if item == 2400:
        departure_time.append(datetime.datetime.strptime('0000', '%H%M').strftime('%H:%M'))
    elif item >= 1000:
        departure_time.append(datetime.datetime.strptime(str(item), '%H%M').strftime('%H:%M'))
    elif item >= 100 and item <= 999:
        departure_time.append(datetime.datetime.strptime('0' + str(item), '%H%M').strftime('%H:%M'))
    elif item >= 10 and item <= 99:
        departure_time.append(datetime.datetime.strptime('00'+ str(item), '%H%M').strftime('%H%M'))
    elif item <=9 :
        departure_time.append(datetime.datetime.strptime('000'+str(item), '%H%M').strftime('%H%M'))
        
wheels_off = []
for item in latedf['WHEELS_OFF'].astype(int):
    if item == 2400:
        wheels_off.append(datetime.datetime.strptime('0000', '%H%M').strftime('%H:%M'))
    elif item >= 1000:
        wheels_off.append(datetime.datetime.strptime(str(item), '%H%M').strftime('%H:%M'))
    elif item >= 100 and item <= 999:
        wheels_off.append(datetime.datetime.strptime('0' + str(item), '%H%M').strftime('%H:%M'))
    elif item >= 10 and item <= 99:
        wheels_off.append(datetime.datetime.strptime('00'+ str(item), '%H%M').strftime('%H%M'))
    elif item <=9 :
        wheels_off.append(datetime.datetime.strptime('000'+str(item), '%H%M').strftime('%H%M'))
        
wheels_on = []
for item in latedf['WHEELS_ON'].astype(int):
    if item == 2400:
        wheels_on.append(datetime.datetime.strptime('0000', '%H%M').strftime('%H:%M'))
    elif item >= 1000:
        wheels_on.append(datetime.datetime.strptime(str(item), '%H%M').strftime('%H:%M'))
    elif item >= 100 and item <= 999:
        wheels_on.append(datetime.datetime.strptime('0' + str(item), '%H%M').strftime('%H:%M'))
    elif item >= 10 and item <= 99:
        wheels_on.append(datetime.datetime.strptime('00'+ str(item), '%H%M').strftime('%H%M'))
    elif item <=9 :
        wheels_on.append(datetime.datetime.strptime('000'+str(item), '%H%M').strftime('%H%M'))
        
arrival_time = []
for item in latedf['ARRIVAL_TIME'].astype(int):
    if item == 2400:
        arrival_time.append(datetime.datetime.strptime('0000', '%H%M').strftime('%H:%M'))
    elif item >= 1000:
        arrival_time.append(datetime.datetime.strptime(str(item), '%H%M').strftime('%H:%M'))
    elif item >= 100 and item <= 999:
        arrival_time.append(datetime.datetime.strptime('0' + str(item), '%H%M').strftime('%H:%M'))
    elif item >= 10 and item <= 99:
        arrival_time.append(datetime.datetime.strptime('00'+ str(item), '%H%M').strftime('%H%M'))
    elif item <=9 :
        arrival_time.append(datetime.datetime.strptime('000'+str(item), '%H%M').strftime('%H%M'))
        
departure_delay = []
arrival_delay = []
total_delay = []

for item in latedf['DEPARTURE_DELAY'].astype(int):
    if item < 0:
        departure_delay.append('-' + str(datetime.timedelta(minutes = (-1 * item)))[:-3])
    else:
        departure_delay.append(str(datetime.timedelta(minutes = item))[:-3])

for item in latedf['ARRIVAL_DELAY'].astype(int):
    if item < 0:
        arrival_delay.append('-' + str(datetime.timedelta(minutes = (-1 * item)))[:-3])
    else:
        arrival_delay.append(str(datetime.timedelta(minutes = item))[:-3])
        
for item in latedf['Total Delay'].astype(int):
    if item < 0:
        total_delay.append('-' + str(datetime.timedelta(minutes = (-1 * item)))[:-3])
    else:
        total_delay.append(str(datetime.timedelta(minutes = item))[:-3])

latedf['DEPARTURE_DELAY']=departure_delay
latedf['ARRIVAL_DELAY'] = arrival_delay
latedf['Total Delay'] = total_delay



'TAXI_OUT', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'TAXI_IN'
taxi_out = []
for item in latedf['TAXI_OUT'].astype(int):
    taxi_out.append(str(datetime.timedelta(minutes = item))[:-3])
    
scheduled_time = []
for item in latedf['SCHEDULED_TIME'].astype(int):
    scheduled_time.append(str(datetime.timedelta(minutes = item))[:-3])
    
elapsed_time = []
for item in latedf['ELAPSED_TIME'].astype(int):
    elapsed_time.append(str(datetime.timedelta(minutes = item))[:-3])
    
air_time = []
for item in latedf['AIR_TIME'].astype(int):
    elapsed_time.append(str(datetime.timedelta(minutes = item))[:-3])
    
taxi_in = []
for item in latedf['TAXI_IN'].astype(int):
    taxi_out.append(str(datetime.timedelta(minutes = item))[:-3])
    
latedf['TAXI_OUT'] = taxi_out
latedf['SCHEDULED_TIME'] = scheduled_time
latedf['ELAPSED_TIME'] = elapsed_time
latedf['AIR_TIME'] = air_time
latedf['TAXI_IN'] = taxi_in
        

In [17]:
latedf['SCHEDULED_ARRIVAL'] = scheduled_arrival
latedf['SCHEDULED_DEPARTURE'] = scheduled_departure
latedf['DEPARTURE_TIME'] = departure_time
latedf['WHEELS_OFF'] = wheels_off
latedf['WHEELS_ON'] = wheels_on
latedf['ARRIVAL_TIME'] = arrival_time

C:\Users\myles\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\myles\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\myles\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

In [18]:
latedf.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,Total Delay
225468,2015,2,16,1,AA,2400,N5ESAA,LAX,DFW,0005,...,147.0,1235,04:43,6.0,04:53,04:49,-0:04,0,0,0:01
225470,2015,2,16,1,DL,2579,N693DL,DEN,ATL,0015,...,135.0,1199,05:24,11.0,05:11,05:35,0:24,0,0,0:39
225472,2015,2,16,1,AA,258,N3DEAA,LAX,MIA,0020,...,254.0,2342,07:50,8.0,08:04,07:58,-0:06,0,0,0:14
225474,2015,2,16,1,US,1905,N173US,SFO,CLT,0025,...,252.0,2296,07:43,32.0,08:11,08:15,0:04,0,0,0:29
225475,2015,2,16,1,AA,1234,N3LVAA,LAS,ORD,0027,...,172.0,1514,06:15,7.0,05:50,06:22,0:32,0,0,0:59


In [19]:
cancel_df = update3_flights[update3_flights['CANCELLED']==1]
cancel_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,Total Delay
225529,2015,2,16,1,US,1852,NaN,BOS,CLT,500,...,NaN,728,NaN,NaN,725,NaN,NaN,0,1,NaN
225580,2015,2,16,1,DL,1780,N967DL,BNA,ATL,525,...,NaN,214,NaN,NaN,735,NaN,NaN,0,1,NaN
225602,2015,2,16,1,US,1909,NaN,BOS,PHL,530,...,NaN,280,NaN,NaN,659,NaN,NaN,0,1,NaN
225665,2015,2,16,1,DL,2079,N329NW,BOS,DTW,545,...,NaN,632,NaN,NaN,759,NaN,NaN,0,1,NaN
225672,2015,2,16,1,WN,346,NaN,BDL,ATL,545,...,NaN,859,NaN,NaN,840,NaN,NaN,0,1,NaN


In [20]:
#Drop columns with NAN
cancel_df2 = cancel_df.drop(columns =['ARRIVAL_TIME','ARRIVAL_DELAY','TAXI_IN','WHEELS_ON','AIR_TIME','DEPARTURE_TIME','Total Delay','ELAPSED_TIME','WHEELS_OFF','TAXI_OUT','DEPARTURE_DELAY'])
cancel_df2
cancel_df3 = cancel_df2.dropna()
cancel_df3.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DIVERTED,CANCELLED
225580,2015,2,16,1,DL,1780,N967DL,BNA,ATL,525,70.0,214,735,0,1
225665,2015,2,16,1,DL,2079,N329NW,BOS,DTW,545,134.0,632,759,0,1
225708,2015,2,16,1,WN,4720,N376SW,BNA,MCO,550,110.0,616,840,0,1
225730,2015,2,16,1,AA,1233,N3DYAA,BOS,DFW,555,274.0,1562,929,0,1
225824,2015,2,16,1,DL,750,N892AT,BOS,JFK,600,80.0,187,720,0,1


In [21]:
#ADD CODE HERE

### Tweet Data (Natalie)

MK NOTES
1. Columns to Drop: 'negativereason', 'negativereason_confidence', 'airline_sentiment_gold', 'negativereason_gold', 'tweet_coord', 'tweet_location', 'user_timezone'
2. Convert Date Column to Only Include Date & Not Time
3. Convert Airline Name to Airline Code

In [22]:
#ADD CODE HERE

## Data Transformation

### Flight Data (Myles & Sedra)

MK NOTES
1. Airport Data - No Changes
2. Airline Data - No Changes
3. Flight Data - Use Group By & Filtering to Create Individual Data Frames for Delayed/Early/Cancelled/Diverted Flights for Each Airline (AIRLINE CODE MUST BE IN EACH DATAFRAME)

In [35]:

#CANCELLED FLIGHTS 
group_cancel = cancel_df3.groupby(['AIRLINE'])
for name, group in group_cancel:
    if name == 'UA':
        UA_cancelled = pd.DataFrame(group)
        
        UA_cancelled = UA_cancelled.reset_index().drop(['index'], axis = 1)
    elif name == 'AA':
        AA_cancelled = pd.DataFrame(group)
        
        AA_cancelled = AA_cancelled.reset_index().drop(['index'],axis = 1)
    elif name == 'DL':
        DL_cancelled = pd.DataFrame(group)
        
        DL_cancelled = DL_cancelled.reset_index().drop(['index'],axis = 1)
    elif name == 'US':
        US_cancelled = pd.DataFrame(group)
        
        US_cancelled = US_cancelled.reset_index().drop(['index'],axis = 1)
    elif name =='WN':
        WN_cancelled = pd.DataFrame(group)
        
        WN_cancelled = WN_cancelled.reset_index().drop(['index'],axis = 1)
    elif name == 'VX':
        VX_cancelled = pd.DataFrame(group)
        
        VX_cancelled = VX_cancelled.reset_index().drop(['index'],axis = 1)
        
print(UA_cancelled)
print(AA_cancelled)
print(DL_cancelled)
print(US_cancelled)
print(WN_cancelled) 
print(VX_cancelled)

AA/DL/US/WN/VX
    
    

   YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER  \
0  2015      2   16            1      UA            627      N818UA   
1  2015      2   16            1      UA           1429      N37470   
2  2015      2   18            3      UA           1159      N13248   
3  2015      2   21            6      UA           1220      N27901   
4  2015      2   21            6      UA            217      N808UA   

  ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE  SCHEDULED_TIME  \
0            DCA                 ORD                 1745           134.0   
1            IAD                 LAS                 1949           327.0   
2            EWR                 SJU                 2000           233.0   
3            DEN                 IAH                 1445           136.0   
4            IAD                 SFO                 1629           369.0   

   DISTANCE  SCHEDULED_ARRIVAL  DIVERTED  CANCELLED  
0       612               1859         0          1  
1 

NameError: name 'AA' is not defined

In [40]:
#Late 
group_late = latedf.groupby(['AIRLINE'])
for name, group in group_cancel:
    if name == 'UA':
        UA_late = pd.DataFrame(group)
        
        UA_late= UA_late.reset_index().drop(['index'], axis = 1)
    elif name == 'AA':
        AA_late = pd.DataFrame(group)
        
        AA_late = AA_late.reset_index().drop(['index'],axis = 1)
    elif name == 'DL':
        DL_late = pd.DataFrame(group)
        
        DL_late = DL_late.reset_index().drop(['index'],axis = 1)
    elif name == 'US':
        US_late = pd.DataFrame(group)
        
        US_late = US_late.reset_index().drop(['index'],axis = 1)
    elif name =='WN':
        WN_late = pd.DataFrame(group)
        
        WN_late = WN_late.reset_index().drop(['index'],axis = 1)
    elif name == 'VX':
        VX_late = pd.DataFrame(group)
        
        VX_late = VX_late.reset_index().drop(['index'],axis = 1)
        
print(UA_late)
print(AA_late)
print(DL_late)
print(US_late)
print(WN_late) 
print(VX_late)



   YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER  \
0  2015      2   16            1      UA            627      N818UA   
1  2015      2   16            1      UA           1429      N37470   
2  2015      2   18            3      UA           1159      N13248   
3  2015      2   21            6      UA           1220      N27901   
4  2015      2   21            6      UA            217      N808UA   

  ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE  SCHEDULED_TIME  \
0            DCA                 ORD                 1745           134.0   
1            IAD                 LAS                 1949           327.0   
2            EWR                 SJU                 2000           233.0   
3            DEN                 IAH                 1445           136.0   
4            IAD                 SFO                 1629           369.0   

   DISTANCE  SCHEDULED_ARRIVAL  DIVERTED  CANCELLED  
0       612               1859         0          1  
1 




### Tweet Data (Natalie)

MK NOTES
1. Use Group By & Filtering to Create Data Frames of Positive/Negative/Neutral Tweets for Each Airline (AIRLINE CODE MUST BE IN EACH DATAFRAME)

In [ ]:
#ADD CODE HERE

## Data Loading (Matt)

MK Notes
1. Create ERD for Completed Database
2. Create Database in PGAdmin
3. Use SQL Alchemy to Load Dataframes into Tables & Relate by Keys

In [ ]:
#ADD CODE HERE

# Project Analysis

## Data Extraction

### Flight Data (Myles & Sedra)

ADD TEXT HERE

### Tweet Data (Natalie)

ADD TEXT HERE

## Data Transformation

### Flight Data (Myles & Sedra)

ADD TEXT HERE

### Tweet Data (Natalie)

ADD TEXT HERE

## Data Loading (Matt)

ADD TEXT HERE